# Delta Table Maintenance

This Notebook is used to perform maintenance tasks on a single Delta Table. For more on this topic check out https://docs.delta.io/latest/delta-utility.html#remove-files-no-longer-referenced-by-a-delta-table

#### Step 1 - Check to see if the table Delta Table exists.
If you see it in the listing, skip to step 3

In [ ]:
%%sql

SHOW TABLES

#### Step 2  - Declare the values to pass on the other cells later in the notebook.
This is the location of your delta table parquet files and the delta_log directory in your ADLS Gen2 account.  
This step is only needed if you delta table did not show in the listing above.

In [ ]:
from pyspark.sql import SparkSession 
from pyspark.sql.types import * 
from delta.tables import DeltaTable
from notebookutils import mssparkutils

# Primary storage info 
account_name = 'demosynapsesilver' # fill in your primary account name 
container_name = 'sales' # fill in your container name
relative_path = 'customers/' # fill in your relative folder path 

adls_path = 'abfss://%s@%s.dfs.core.windows.net/%s' % (container_name, account_name, relative_path) 
print('Primary storage account path: ' + adls_path) 
spark.conf.set("sql.adls_path", adls_path) #Used for Spark SQL commands in later cells

Shows the files in your ADLS Gen 2 account.  This verifies that the path above is correct.

In [ ]:
files = mssparkutils.fs.ls(adls_path)
for file in files:
    print(file.path, file.size)

#### Step 3 - Declare our Delta Table values
If your Delta Table Name was in the listing of step 1, add it below, replacing the SilverCustomers value.  Otherwise declare your Delta table as needed.  

In [ ]:
# Delta Table Settings
spark.conf.set("sql.delta_table", 'SilverCustomers') #Delta Table Name
spark.conf.set("sql.vacuum_hours", 168)  # Vacuum setting for the Delta Table - 168 hours equals 1 week

#### Step 4 - Create the Delta table if it did not show in the listing above
Skip this step if your Delta Table Name was in the listing of step 1.

In [ ]:
%%sql 

CREATE TABLE IF NOT EXISTS ${sql.delta_table}    
USING DELTA
LOCATION '${sql.adls_path}';

#### Step 5 - Get a the properties of the Delta Table
This is a verification step

In [ ]:
%%sql

DESCRIBE DETAIL ${sql.delta_table}	

#### Step 6 - Optimize Table Command
You can add more options to the command.  For more on this topic check out https://learn.microsoft.com/en-us/azure/databricks/sql/language-manual/delta-optimize

In [ ]:
%%sql

OPTIMIZE ${sql.delta_table};

#### Step 7 -  Vacuum table
You can add more options to command.  For more on this topic check out https://learn.microsoft.com/en-us/azure/databricks/delta/vacuum

In [ ]:
%%sql

VACUUM ${sql.delta_table} RETAIN ${sql.vacuum_hours} HOURS

#### Step 8 - Shows the files in Storage after Vacuum Command
This is a verification step

In [ ]:
files = mssparkutils.fs.ls(adls_path)
for file in files:
    print(file.path, file.size)